# SQL Servers metadata

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-02-05cs

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om system-databaser i SQL Server hos Microsoft:

- [https://learn.microsoft.com/en-us/sql/relational-databases/databases/system-databases](https://learn.microsoft.com/en-us/sql/relational-databases/databases/system-databases?view=sql-server-ver16)

Læs mere om metadata i `INFORMATION_SCHEMA`-views (ANSI/ISO standard kataloger for metadata) i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/relational-databases/system-information-schema-views/system-information-schema-views-transact-sql](https://learn.microsoft.com/en-us/sql/relational-databases/system-information-schema-views/system-information-schema-views-transact-sql?view=sql-server-ver16)

Læs også om den udvidede metadata i `sys`-views (Object Catalog Views) i Microsofts T-SQL reference:
- [https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/object-catalog-views-transact-sql](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/object-catalog-views-transact-sql?source=recommendations&view=sql-server-ver16)

## Indholdsfortegnelse 

- [System-databaser i SQL Server](#systemdatabaser)
- [INFORMATION_SCHEMA-views](#INFORMATION_SCHEMA-views)
- [sys-views](#sys-views)
- [Hovedpointer](#Hovedpointer)

## System-databaser i SQL Server <a class="anchor" id="systemdatabaser"></a>

Ud over de databaser, som indeholder data, som vi som brugere lægger i SQL Server, findes der altid fire databaser i SQL Server:

- `master`
- `model`
- `msdb`
- `tempdb`

`master`-databasen indeholder al den vigtige information, som får databaseserveren til at virke, herunder metadata om de databaser som findes i databaseserveren og security-oplysningerne, som definerer hvem der må tilgå databaserne og hvordan. 

`model`-databasen er en skabelonsdatabase, som nye databaser oprettes udfra.

`msdb`-databasen indeholder data fra SQL Agent-jobs.

`tempdb`-databasen er arbejdsområde for temporære objekter og mellemresultater mv. Denne database sættes typisk til at have en startstørrelse og til at kunne vokse dynamisk ved behov. Hvis en forespørgsel kræver mere plads, end `tempdb`-databasen indeholder, vil den derfor øges. Men databasen mindskes ikke igen, før den eksplicit bliver reduceret med en `shrink`-operation, se [https://learn.microsoft.com/en-us/sql/relational-databases/databases/shrink-a-database](https://learn.microsoft.com/en-us/sql/relational-databases/databases/shrink-a-database?redirectedfrom=MSDN&view=sql-server-ver16).

## `INFORMATION_SCHEMA`-views

`INFORMATION_SCEMA` er et systemskema i hver database, som indeholder metadata i ANSI/ISO standard format.

Oplysningerne bruges bl.a. til at danne et overblik over indhold og til at automatisere databaseopgaver som indebærer mange gentagelser.

De mest anvendte views i `INFORMATION_SCHEMA`-skemaet er:

```
COLUMNS                 -- Kolonner i tabeller og views
TABLES                  -- Tabeller og views 
VIEWS                   -- Views m. view-defintion
ROUTINES                -- Stored Procedures og funktioner 
VIEW_COLUMN_USAGE       -- Views som peger på COLUMNs i andre tabeller og views
VIEW_TABLE_USAGE        -- Views som peger på andre tabeller og views
REFERENTIAL_CONSTRAINTS -- Referencer mellem fremmed- og primærnøgler i tabeller, men tabel- og kolonnenavne ikke med
KEY_COLUMN_USAGE        -- Sammenhæng mellem nøgle (constraint) og tabeller og kolonner
```

In [ ]:
SELECT * FROM INFORMATION_SCHEMA.COLUMNS;                 -- Kolonner i tabeller og views
SELECT * FROM INFORMATION_SCHEMA.TABLES;                  -- Tabeller og views 
SELECT * FROM INFORMATION_SCHEMA.VIEWS;                   -- Views m. view-defintion
SELECT * FROM INFORMATION_SCHEMA.ROUTINES;                -- Stored procedures og funktioner.
SELECT * FROM INFORMATION_SCHEMA.VIEW_COLUMN_USAGE;       -- Views som peger på COLUMNs i andre tabeller og views
SELECT * FROM INFORMATION_SCHEMA.VIEW_TABLE_USAGE;        -- Views som peger på andre tabeller og views
SELECT * FROM INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS; -- Referencer mellem fremmed- og primærnøgler i tabeller, men tabel- og kolonnenavne ikke med
SELECT * FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE;        -- Sammenhæng mellem nøgle (constraint) og tabeller og kolonner

In [ ]:
/* Når vi bruger `REFERENTIAL_CONSTRAINTS`-viewet som brotabel mellem et `KEY_COLUMN_USAGE`-view
    som fra-tabel og et `KEY_COLUMN_USAGE`-view som til-tabel, så kan vi lave en samlet
    forespørgsel, som viser fremmednøgler-kolonner i en fra-tabel og de tilhørende
    primærnøgle-kolonner i en til-tabel: */

SELECT 
     f.CONSTRAINT_NAME AS FK_CONSTRAINT_NAME 
    ,f.TABLE_SCHEMA AS FK_TABLE_SCHEMA 
    ,f.TABLE_NAME AS FK_TABLE_NAME 
    ,f.COLUMN_NAME AS FK_COLUMN_NAME 
    ,t.CONSTRAINT_NAME AS REF_CONSTRAINT_NAME 
    ,t.TABLE_SCHEMA AS REF_TABLE_SCHEMA 
    ,t.TABLE_NAME AS REF_TABLE_NAME 
    ,t.COLUMN_NAME AS REF_COLUMN_NAME 
FROM INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS AS b 
INNER JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS f 
    ON f.CONSTRAINT_CATALOG = b.CONSTRAINT_CATALOG 
    AND f.CONSTRAINT_SCHEMA = b.CONSTRAINT_SCHEMA 
    AND f.CONSTRAINT_NAME = b.CONSTRAINT_NAME 
INNER JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS t 
    ON t.CONSTRAINT_CATALOG = b.UNIQUE_CONSTRAINT_CATALOG 
    AND t.CONSTRAINT_SCHEMA = b.UNIQUE_CONSTRAINT_SCHEMA 
    AND t.CONSTRAINT_NAME = b.UNIQUE_CONSTRAINT_NAME;


## `sys`-views

`sys`-views er SQL Servers egne views med et rigere tilbud af metadata. Der er her endnu flere views og endnu flere informationer. 

Mange udviklere synes at views i `sys`-skemaet er sværere at bruge, da de er mere normaliserede end views i `INFORMATION_SCHEMA`. 

`sys`-schemaet indeholder bl.a.:

```
sys.columns
sys.tables
sys.views
sys.procedures
sys.foreign_key_columns
sys.identity_columns
sys.index_columns
(39 views i alt)
```


Sammenhængen mellem `INFORMATION_SCHEMA`- og `sys`-views bliver klar, når vi finder kildekoden for views i `INFORMATION_SCHEMA`. Fx finder vi kildekoden til `INFORMATION_SCHEMA.VIEWS` som:

```sql
SELECT OBJECT_DEFINITION (OBJECT_ID(N'INFORMATION_SCHEMA.VIEWS'));
```

og får svaret, at `INFORMATION_SCHEMA.VIEWS` er et view bygget på `sys.views`:

```sql
CREATE VIEW INFORMATION_SCHEMA.VIEWS AS 
SELECT 	
  DB_NAME() AS TABLE_CATALOG
  , SCHEMA_NAME(schema_id) AS TABLE_SCHEMA
  , name AS TABLE_NAME
  , convert(nvarchar(4000),OBJECT_DEFINITION(object_id)) AS VIEW_DEFINITION
  , convert(varchar(7), CASE with_check_option WHEN 1 THEN 'CASCADE' ELSE 'NONE' END) AS CHECK_OPTION
  , 'NO' AS IS_UPDATABLE 
FROM sys.views 
```


### *Tid til opgaver...*

Lav opgave 1 og 2 i [opgavehæftet](QueryingMetadata.sql).

## Hovedpointer

- Foruden de brugeroprettede databaser, findes i SQL Server fire databaser, som selve databaseserveren anvender:
  - `master`
  - `model`
  - `msdb`
  - `tempdb`
- Metadata om de brugerdefinerede databasers indhold findes i to versioner
  - `INFORMATION_SCHEMA` med ANSI/ISO standard metadata, som er nemt overskueligt
  - `sys` med SQL Servers egen metadata med langt flere detaljer
- Views i `INFORMATION_SCHEMA` er scripts mod information i `sys`!

## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
